**Цели:**
* Изучить, сколько книг вышло после 1 января 2000 года;
* Изучить для каждой книги количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
* Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Изучить среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})



In [2]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Изучим, сколько книг вышло после 1 января 2000 года.

In [3]:
df_1 = '''SELECT COUNT(book_id) as всего_книг
          FROM books
          WHERE CAST(publication_date AS date) > '2000-01-01' '''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(df_1), con = con))

,всего_книг
0,819


Всего вышло после 1 января 2000 года `819 книг`.

Изучим для каждой книги количество обзоров и среднюю оценку.

In [4]:
df_2 = '''  
            SELECT b.title as название_книги,
                    COUNT(DISTINCT review_id) as количество_обзоров,
                    ROUND(AVG (rating), 2) as средний_рейтинг
            FROM books AS b 
             LEFT JOIN ratings AS r ON b.book_id = r.book_id
             LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
            GROUP BY b.book_id
                    ORDER BY средний_рейтинг DESC, 
                 количество_обзоров DESC
           '''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(df_2), con = con))

,название_книги,количество_обзоров,средний_рейтинг
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


У нас имеется 1000 книг, отсортированные по средней оценке и количеству обзоров. Книга с наибольшим рейтингом и количеством обзоров (среди средних оценок 5) `-A Dirty Job (Grim Reaper #1)`.

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [5]:
df_3 = '''SELECT p.publisher as издатель
          FROM books as b
          JOIN publishers AS p ON b.publisher_id = p.publisher_id
          WHERE b.num_pages > 50
          GROUP BY p.publisher
          ORDER BY COUNT(b.book_id) DESC
          LIMIT 1'''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(df_3), con = con))

,издатель
0,Penguin Books


Издательство, которое выпустило наибольшее число книг толще 50 страниц является `Penguin Books`.

Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками.

In [6]:
df_4 = '''WITH q AS (SELECT b.book_id,
                            COUNT(r.rating_id) as count_rating,
                            ROUND(AVG (r.rating), 2) as avg_ratings,
                            b.author_id
                    FROM books AS b 
                    JOIN ratings AS r ON b.book_id = r.book_id
                    GROUP BY b.book_id
                   ), 
               q1 AS( 
                      SELECT *
                      FROM q
                      JOIN authors AS a ON q.author_id = a.author_id
                      WHERE q.count_rating > 50
                      ORDER BY q.avg_ratings DESC
                      )
                  SELECT author,
                          AVG(avg_ratings)
                  FROM q1
                  GROUP BY author
                  LIMIT 1
          '''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(df_4), con = con))

,author,avg
0,J.K. Rowling/Mary GrandPré,4.285


Автором с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками является `J.K. Rowling/Mary GrandPré`.

Изучим среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [7]:
df_5 = '''SELECT ROUND(AVG(review_cnt)) AS среднее_количество_обзоров
          FROM (
                   SELECT username, 
                          COUNT(review_id) AS review_cnt
                   FROM reviews 
                   WHERE username IN (
                                      SELECT username
                                      FROM ratings
                                      GROUP BY username
                                      HAVING COUNT(rating_id) > 50)
                   GROUP BY  username
                  ) AS rereview_count
            '''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(df_5), con = con))

,среднее_количество_обзоров
0,24.0


Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет `24`.

**Вывод**
- Всего вышло после 1 января 2000 года `819 книг`.
- У нас имеется 1000 книг, отсортированные по средней оценке и количеству обзоров. Книга с наибольшим рейтингом и количеством обзоров (среди средних оценок 5) `-A Dirty Job (Grim Reaper #1)`.
- Издательство, которое выпустило наибольшее число книг толще 50 страниц является `Penguin Books`.
-Автором с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками является `J.K. Rowling/Mary GrandPré`.
- Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет `24`.